In [1]:
# import libraries
import pandas as pd
import numpy as np

# get the data from csv file to dataframe
raw_data = pd.read_csv('amazon.csv')
# check column name
raw_data.columns

# dataframe kebutuhan
data_column=['userId', 'itemId', 'rating']
data = pd.DataFrame(columns=data_column)



#START DATA MUNGING
# knowing the mess at column 21-23 
# rename the column
raw_data.rename( columns={'Unnamed: 21':'nan1', 'Unnamed: 22':'nan2', 'Unnamed: 23':'nan3' }, inplace=True )

# solving the first problem
raw_data.loc[pd.notnull(raw_data['nan3']), 'reviews.username'] = raw_data.nan3
raw_data.loc[pd.notnull(raw_data['nan1']), 'reviews.username'] = raw_data.nan1
raw_data.loc[pd.notnull(raw_data['nan2']), 'reviews.username'] = raw_data.nan2

/home/dea/Documents/projects/RecSys/recenv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1,10,19,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
# solving the second problem
# check unique username and goods name
user_count = raw_data['reviews.username'].unique().shape[0]
item_count = raw_data['name'].unique().shape[0]
# indexing username
username = []
user_id = []
start_id = 1

for x in raw_data['reviews.username']:
    if username:
        for y in username:
            if x not in username :
                username.append(x)
                user_id.append(start_id)
                start_id = start_id + 1
            else:
                break
    else:
        username.append(x)
        user_id.append(start_id)
        start_id = start_id + 1

user_table = {
    'userId':user_id,
    'username':username
}
user_df = pd.DataFrame(user_table)
# check the user_df
# user_df

# indexing item
itemname = []
item_id = []
start_id = 1

for x in raw_data['name']:
    if itemname:
        for y in itemname:
            if x not in itemname :
                itemname.append(x)
                item_id.append(start_id)
                start_id = start_id + 1
            else:
                break
    else:
        itemname.append(x)
        item_id.append(start_id)
        start_id = start_id + 1

item_table = {
    'itemId':item_id,
    'itemname':itemname
}
item_df = pd.DataFrame(item_table)
# check the item_df
item_df.head(50)

,itemId,itemname
0,1,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,..."
1,2,Kindle Oasis E-reader with Leather Charging Co...
2,3,"Amazon Kindle Lighted Leather Cover,,,\nAmazon..."
3,4,"Amazon Kindle Lighted Leather Cover,,,\nKindle..."
4,5,"Kindle Keyboard,,,\nKindle Keyboard,,,"
5,6,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,..."
6,7,"Fire HD 8 Tablet with Alexa, 8 HD Display, 32 ..."
7,8,Amazon 5W USB Official OEM Charger and Power A...
8,9,"All-New Kindle E-reader - Black, 6 Glare-Free ..."
9,10,"Amazon Kindle Fire Hd (3rd Generation) 8gb,,,\..."


In [3]:
# check unique username and item name
user_count = raw_data['reviews.username'].unique().shape[0]
item_count = raw_data['name'].unique().shape[0]

#the user_df has amount as user_count
#but the item_df hasnot same amount as item_count
#resolving with :

item_df= item_df.dropna()
#check unique item name again
check_item = item_df['itemname'].unique().shape[0]

In [4]:
raw_data = pd.merge(raw_data, user_df, how='left',
                     left_on='reviews.username', right_on='username')

In [6]:
raw_data = pd.merge(raw_data, item_df, how='left',
                     left_on='name', right_on='itemname')

In [7]:
raw_data 

,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.userCity,reviews.userProvince,reviews.username,nan1,nan2,nan3,userId,username,itemId,itemname
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,NaN,NaN,Adapter,NaN,NaN,NaN,1,Adapter,1.0,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,..."
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,NaN,NaN,truman,NaN,NaN,NaN,2,truman,1.0,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,..."
2,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,NaN,NaN,DaveZ,NaN,NaN,NaN,3,DaveZ,1.0,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,..."
3,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,NaN,NaN,Shacks,NaN,NaN,NaN,4,Shacks,1.0,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,..."
4,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-12T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,NaN,NaN,explore42,NaN,NaN,NaN,5,explore42,1.0,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,..."
5,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-12T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,NaN,NaN,tklit,NaN,NaN,NaN,6,tklit,1.0,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,..."
6,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-12T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,NaN,NaN,Droi,NaN,NaN,NaN,7,Droi,1.0,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,..."
7,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-12T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,NaN,NaN,Kacy,NaN,NaN,NaN,8,Kacy,1.0,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,..."
8,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-23T00:00:00.000Z,2017-07-03T23:28:24Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:44:00.000Z",...,NaN,NaN,Weebee,NaN,NaN,NaN,9,Weebee,1.0,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,..."
9,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-23T00: